In [ ]:
!pip install tensorflow-gpu==1.13.1

     |████████████████████████████████| 345.0 MB 3.6 kB/s 
     |████████████████████████████████| 3.2 MB 41.8 MB/s 
     |████████████████████████████████| 367 kB 75.8 MB/s 
     |████████████████████████████████| 50 kB 6.6 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.7.0
    Uninstalling tensorflow-estimator-2.7.0:
      Successfully uninstalled tensorflow-estimator-2.7.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.7.0
    Uninstalling tensorboard-2.7.0:
      Successfully uninstalled tensorboard-2.7.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.7.0 requires tensorboard~=2.6, but you have tensorboard 1.13.1 which is incompatible.
tensorflow 2.7.0 requires tensorflow-estimator<2.8,~=2.7.0rc0, but you have tensorflow-estimator 1.13.0 which is in

In [ ]:
!pip install tensorflow==1.13.1

In [ ]:
import os
import numpy as np
import tensorflow as tf

import BKNetStyle
from const import *
import time
import math
%load_ext autoreload
%autoreload 2
from copy import copy, deepcopy

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
NUM_CLASSES = 3

def normalise(x):
    return (x-128)/255.0

def load_mini_dataset(index):
    
    label_to_int = {"Mask":0,"Mask_Mouth_Chin":1,"Mask_Chin":2,"Mask_Nose_Mouth":3}
    int_to_label = {0:"Mask",1:"Mask_Mouth_Chin",2:"Mask_Chin",3:"Mask_Nose_Mouth"}


    filename_prefix = '/content/sample_data/test_part'#/content/drive/MyDrive/GT/BigDataProject/train_part0.npy
    filename = filename_prefix + str(index) + ".npy"
                
    X = np.load(filename, allow_pickle=True)
  
    x_mini_data =[]
    y_mini_data=[]
    mask = []    
    mask_prim = 0.0
    mask_sec = 1.0

    for sample in X:
        
        norm_x_prim = normalise(sample[0]) # todo check output once

        label_int = sample[1]
        y_prim = [0] * NUM_CLASSES

        if int_to_label[label_int]=="Mask":
          x_mini_data.append(norm_x_prim)
          y_prim[label_int] = 1 # y_ = [1,0,0]
          y_mini_data.append(y_prim)
          mask.append(mask_prim)
          
        else:
          x_mini_data.append(norm_x_prim)
          y_prim[1] = 1 # y_ = [0,1,0]
          y_mini_data.append(y_prim)
          mask.append(mask_prim)

          # create a copy for x & y, add same x and new y for secondary task
          norm_x_sec = deepcopy(norm_x_prim)
          y_sec = [0] * NUM_CLASSES
          y_sec[label_int-1] = 1
          
          x_mini_data.append(norm_x_sec)
          y_mini_data.append(y_sec)

          mask.append(mask_sec)
          

    return x_mini_data, y_mini_data,mask


def create_random_mini_batches(X, Y, mask_mini, mini_batch_size = 32):
    
    m = len(X) # number of training examples
    mini_batches = []
    
    #Reshaping to convert Y to a 2D array from a rank one array
    X = np.array(X)
    Y = np.array(Y)
    Z = np.array(mask_mini)
    
    #Shuffle the data in each of the mini batch
    permutation = list(np.random.permutation(m))
    shuffled_X = X#[permutation, :]
    shuffled_Y = Y#[permutation, :]
    shuffled_Z = Z#[permutation]
    
    n_mini_batches = math.ceil(m / mini_batch_size) # number of mini batches of size mini_batch_size in your partitionning
    for k in range(n_mini_batches):
        
        start_pos = k * mini_batch_size
        end_pos = min(start_pos + mini_batch_size, m)
        
        mini_batch_X = shuffled_X[start_pos : end_pos, :]
        mini_batch_Y = shuffled_Y[start_pos : end_pos, :]
        mini_batch_Z = shuffled_Z[start_pos : end_pos]
        
        mini_batch = (mini_batch_X, mini_batch_Y,mini_batch_Z)
        mini_batches.append(mini_batch)

    return mini_batches


In [ ]:
import numpy as np

def one_hot(indices, num_classes):
    tmp = np.zeros((indices.shape[0], num_classes), dtype=np.float32)
    tmp[np.arange(indices.shape[0]), indices] = 1.0
    return tmp



[  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107]


In [ ]:
sess = tf.compat.v1.InteractiveSession()
global_step = tf.contrib.framework.get_or_create_global_step()

x, y_, mask = BKNetStyle.Input()

y_prim_conv, y_sec_conv, phase_train, keep_prob = BKNetStyle.BKNetModel(x)


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please switch to tf.train.get_or_create_global_step
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [ ]:
prim_loss, sec_loss, l2_loss, loss = BKNetStyle.selective_loss(y_prim_conv, y_sec_conv, y_, mask)

train_step = BKNetStyle.train_op(loss, global_step)

prim_mask = tf.get_collection('prim_mask')[0]
sec_mask = tf.get_collection('sec_mask')[0]

y_prim = tf.get_collection('y_prim')[0]
y_sec = tf.get_collection('y_sec')[0]

prim_correct_prediction = tf.equal(tf.argmax(y_prim_conv, 1), tf.argmax(y_prim, 1))
sec_correct_prediction = tf.equal(tf.argmax(y_sec_conv, 1), tf.argmax(y_sec, 1))

prim_true_pred = tf.reduce_sum(tf.cast(prim_correct_prediction, dtype=tf.float32) * prim_mask)
sec_true_pred = tf.reduce_sum(tf.cast(sec_correct_prediction, dtype=tf.float32) * sec_mask)

saver = tf.train.Saver()
print('Restoring existed model')
saver.restore(sess, "/content/models/" + 'model.ckpt')
print('OK')

loss_summary_placeholder = tf.placeholder(tf.float32)
tf.summary.scalar('loss', loss_summary_placeholder)
merge_summary = tf.summary.merge_all()
writer = tf.summary.FileWriter("./summary/")

avg_prim_loss = []
avg_sec_loss = []

prim_nb_true_pred = 0
sec_nb_true_pred = 0

prim_nb_train = 0
sec_nb_train = 0

    #added by Samarth; lazy loading of dataset and training
n_mini_batches = 0
epoch_cost = 0
total_time_taken_to_load_dataset = 0
total_time_taken_to_create_mini_batches = 0            
tic = time.time()
#file_indices = get_shuffled_indices(1, 26) # todo change to total part files
file_indices = np.arange(0, 27) #[0, 1, 2] #TODO - to remove

mini_batch_size = BATCH_SIZE

true_pos_count = 0
false_pos_count = 0
false_neg_count = 0
true_neg_count = 0
TP = tf.count_nonzero(tf.cast(tf.argmax(y_prim_conv, 1) * tf.argmax(y_prim, 1), dtype=tf.float32) * prim_mask, axis=0)
FP = tf.count_nonzero(tf.cast(tf.argmax(y_prim_conv, 1) * (tf.argmax(y_prim, 1) - 1), dtype=tf.float32) * prim_mask, axis=0)
FN = tf.count_nonzero(tf.cast((tf.argmax(y_prim_conv, 1) - 1) * tf.argmax(y_prim, 1), dtype=tf.float32) * prim_mask, axis=0)
TN = tf.count_nonzero(tf.cast((tf.argmax(y_prim_conv, 1) - 1) * (tf.argmax(y_prim, 1) - 1), dtype=tf.float32) * prim_mask, axis=0)

for j in file_indices:
    
    X_mini = None
    Y_mini = None

    lmd_tic = time.time()
    print("at file index ", j)
    X_mini, Y_mini, mask_mini = load_mini_dataset(j) # todo Samarth add for mask - done
    #print("at file index ", j)
    lmd_toc = time.time()
    total_time_taken_to_load_dataset += (lmd_toc-lmd_tic)
    #print(total_time_taken_to_load_dataset)
    
    rmb_tic = time.time()
    mini_batches = None
    mini_batches = create_random_mini_batches(X_mini, Y_mini, mask_mini, mini_batch_size = mini_batch_size) # todo Samarth add for mask
    rmb_toc = time.time()
    total_time_taken_to_create_mini_batches += (rmb_toc-rmb_tic)
    
    
    '''for batch in range(number_batch):
    #       print('Training on batch {0}/{1}'.format(str(batch + 1), str(number_batch)))
    top = batch * BATCH_SIZE
    bot = min((batch + 1) * BATCH_SIZE, len(train_data))
    batch_img = np.asarray(train_img[top:bot])
    batch_label = np.asarray(train_label[top:bot])
    batch_mask = np.asarray(train_mask[top:bot])'''
    #bat_ind = 0
    for mini_batch in mini_batches:
        #print("at minibatch ", bat_ind)
        #bat_ind+=1
        batch_img = mini_batch[0] 
        batch_label = mini_batch[1]
        batch_mask = mini_batch[2]
        
        # (batch_img, batch_label) = mini_batch
        curr_batch_size = batch_img.shape[0]
        # batch_mask = np.zeros(curr_batch_size) #todo comment Samarth - done

        for i in range(curr_batch_size):
            if batch_mask[i] == 0.0:
                prim_nb_train += 1
            else:
                if batch_mask[i] == 1.0:
                    sec_nb_train += 1                        
                # else:
                #     age_nb_train += 1

        #batch_img = CNN2Head_input.augmentation(batch_img, 48)

        '''ttl, pml, sel, l2l, _ = sess.run([loss, prim_loss, sec_loss, l2_loss, train_step],
                                              feed_dict={x: batch_img, y_: batch_label, mask: batch_mask,
                                                        phase_train: True,
                                                        keep_prob: 0.5})'''
        #t = lmd_tic = time.time()
        prim_nb_true_pred += sess.run(prim_true_pred, feed_dict={x: batch_img, y_: batch_label, mask: batch_mask,
                                                                  phase_train: False,
                                                                  keep_prob: 0.5})
        #print('time taken', time.time() - t)
        sec_nb_true_pred += sess.run(sec_true_pred,
                                        feed_dict={x: batch_img, y_: batch_label, mask: batch_mask,
                                                  phase_train: False,
                                                  keep_prob: 0.5})
        
        true_pos, false_pos, false_neg, true_neg = sess.run([TP, FP, FN, TN], feed_dict={x: batch_img, y_: batch_label, mask: batch_mask,
                                                  phase_train: False,
                                                  keep_prob: 0.5})
        true_pos_count += true_pos
        false_pos_count += false_pos
        false_neg_count += false_neg
        true_neg_count += true_neg
        #print(prim_nb_true_pred, sec_nb_true_pred, prim_nb_train, sec_nb_train)

        # age_nb_true_pred += sess.run(age_true_pred,
        #                                 feed_dict={x: batch_img, y_: batch_label, mask: batch_mask,
        #                                             phase_train: True,
        #                                             keep_prob: 0.5})

        '''--------------------------------------- DEBUG -----------------------------------------------------'''
        '''
        sm_mask, em_mask, ge_mask = sess.run([smile_mask, gender_mask, age_mask],
                                            feed_dict={x: batch_img, y_: batch_label, mask: batch_mask,
                                                        phase_train: True,
                                                        keep_prob: 0.5})
        print('Smile mask: ', sm_mask)
        print('Gender mask', ge_mask)
        print('Age mask', ag_mask)
        print('Batch mask', batch_mask)

        y_true_sm, y_true_ge, y_true_ag = sess.run([y_smile, y_gender, y_age],
                                                  feed_dict={x: batch_img, y_: batch_label, mask: batch_mask,
                                                              phase_train: True,
                                                              keep_prob: 0.5})
        print('Smile label', y_true_sm)
        print('Gender label', y_true_ge)
        print('Age label', y_true_ag)
        print('Batch label', batch_label)

        y_conv_sm, y_conv_ge, y_conv_ag = sess.run([y_smile_conv, y_gender_conv, y_age_conv],
                                                  feed_dict={x: batch_img, y_: batch_label, mask: batch_mask,
                                                              phase_train: True,
                                                              keep_prob: 0.5})
        print('Smile conv', y_conv_sm)
        print('Gender conv', y_conv_ge)
        print('Age conv', y_conv_ag)

        '''
        '''---------------------------------- END OF DEBUG ----------------------------------------------------'''

            


prim_train_accuracy = prim_nb_true_pred * 1.0 / prim_nb_train
sec_train_accuracy = sec_nb_true_pred * 1.0 / sec_nb_train
precision_class0 = true_pos_count / (true_pos_count + false_pos_count)
recall_class0 = true_pos_count / (true_pos_count + false_neg_count)
f1_class0 = 2 * precision_class0 * recall_class0 / (precision_class0 + recall_class0)
print(true_pos_count, false_pos_count, false_neg_count, true_neg_count)

precision_class1 = true_neg_count / (true_neg_count + false_neg_count)
recall_class1 = true_neg_count / (true_neg_count + false_pos_count)
f1_class1 = 2 * precision_class1 * recall_class1 / (precision_class1 + recall_class1)
    # age_train_accuracy = age_nb_true_pred * 1.0 / age_nb_train


#     print('Avg_ttl: ' + str(avg_ttl))
#     print('loss_summary_placeholder: ' + str(loss_summary_placeholder))
#     print('merge_summary: ' + str(merge_summary))

    #summary = sess.run(merge_summary, feed_dict={loss_summary_placeholder: avg_ttl})
    #writer.add_summary(summary, global_step=epoch)
    
    #with open('log.csv', 'w+') as f:
        # epochs, smile_train_accuracy, gender_train_accuracy, age_train_accuracy,
        # avg_smile_loss, avg_gender_loss, avg_age_loss, avg_ttl, avg_rgl
        # f.write('{0},{1},{2},{3},{4},{5},{6},{7},{8}\n'.format(current_epoch, smile_train_accuracy, gender_train_accuracy, age_train_accuracy, avg_smile_loss, avg_gender_loss, avg_age_loss, avg_ttl, avg_rgl))
        #f.write('{0},{1},{2},{3},{4},{5},{6}\n'.format(current_epoch, prim_train_accuracy, sec_train_accuracy, avg_prim_loss, avg_sec_loss, avg_ttl, avg_rgl))

print('Primary task train accuracy: ' + str(prim_train_accuracy * 100))
print('Secondary task train accuracy: ' + str(sec_train_accuracy * 100))
print('Precision: ', precision_class0)
print('Recall: ', recall_class0)
print('F1 score: ', f1_class0)
print('Precision: ', precision_class1)
print('Recall: ', recall_class1)
print('F1 score: ', f1_class1)
    # print('Age task train accuracy: ' + str(age_train_accuracy * 100))

    #print('Total loss: ' + str(avg_ttl) + '. L2-loss: ' + str(avg_rgl))
    #print('Primary loss: ' + str(avg_prim_loss))
    #print('Secondary loss: ' + str(avg_sec_loss))
    # print('Age loss: ' + str(avg_age_loss))
    
print('\n')

    #saver.save(sess, 'sample_data/' + 'model.ckpt')

Restoring existed model
INFO:tensorflow:Restoring parameters from /content/models/model.ckpt
OK
at file index  0
at file index  1
at file index  2
at file index  3
at file index  4
at file index  5
at file index  6
at file index  7
at file index  8
at file index  9
at file index  10
at file index  11
at file index  12
at file index  13
at file index  14
at file index  15
at file index  16
at file index  17
at file index  18
at file index  19
at file index  20
at file index  21
at file index  22
at file index  23
at file index  24
at file index  25
at file index  26
13275 127 68 13287
Primary task train accuracy: 99.29738012482716
Secondary task train accuracy: 98.77089110394964
Precision:  0.9905238024175497
Recall:  0.9949036948212546
F1 score:  0.9927089175546832
Precision:  0.9949082740546612
Recall:  0.990532279707768
F1 score:  0.9927154544435728




In [ ]:

# import tensorflow as tf
# vector_zero = tf.constant(0., tf.float32, [100])
# print(vector_zero) 
# a   c       
#tf.config.list_physical_devices('GPU')  
    
from tensorflow.python.client import device_lib
 
def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    print(local_device_protos)
    return [x.name for x in local_device_protos if x.device_type == 'GPU']
  
print(get_available_gpus())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 4306342662559310609
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 5937106106890450757
physical_device_desc: "device: XLA_CPU device"
]
[]


Saving a copy of misclassified images

In [ ]:
from PIL import Image
import cv2

In [ ]:

prim_loss, sec_loss, l2_loss, loss = BKNetStyle.selective_loss(y_prim_conv, y_sec_conv, y_, mask)

train_step = BKNetStyle.train_op(loss, global_step)

prim_mask = tf.get_collection('prim_mask')[0]
sec_mask = tf.get_collection('sec_mask')[0]

y_prim = tf.get_collection('y_prim')[0]
y_sec = tf.get_collection('y_sec')[0]
 
prim_correct_prediction = tf.equal(tf.argmax(y_prim_conv, 1), tf.argmax(y_prim, 1))
sec_correct_prediction = tf.equal(tf.argmax(y_sec_conv, 1), tf.argmax(y_sec, 1))

prim_true_pred = tf.reduce_sum(tf.cast(prim_correct_prediction, dtype=tf.float32) * prim_mask)
sec_true_pred = tf.reduce_sum(tf.cast(sec_correct_prediction, dtype=tf.float32) * sec_mask)

saver = tf.train.Saver()
print('Restoring existed model')
saver.restore(sess, "/content/models/" + 'model.ckpt')
print('OK')

loss_summary_placeholder = tf.placeholder(tf.float32)
tf.summary.scalar('loss', loss_summary_placeholder)
merge_summary = tf.summary.merge_all()
writer = tf.summary.FileWriter("./summary/")

avg_prim_loss = []
avg_sec_loss = []

prim_nb_true_pred = 0
sec_nb_true_pred = 0

prim_nb_train = 0
sec_nb_train = 0

n_mini_batches = 0
epoch_cost = 0
total_time_taken_to_load_dataset = 0
total_time_taken_to_create_mini_batches = 0            
tic = time.time()

file_indices = np.arange(20) #[0, 1, 2] #TODO - to remove

mini_batch_size = 1

true_pos_count = 0
false_pos_count = 0
false_neg_count = 0
true_neg_count = 0
TP = tf.cast(tf.argmax(y_prim_conv, 1) * tf.argmax(y_prim, 1), dtype=tf.float32) * prim_mask
FP = tf.cast(tf.argmax(y_prim_conv, 1) * (tf.argmax(y_prim, 1) - 1), dtype=tf.float32) * prim_mask
FN = tf.cast((tf.argmax(y_prim_conv, 1) - 1) * tf.argmax(y_prim, 1), dtype=tf.float32) * prim_mask
TN = tf.cast((tf.argmax(y_prim_conv, 1) - 1) * (tf.argmax(y_prim, 1) - 1), dtype=tf.float32) * prim_mask

op = tf.argmax(y_prim_conv, 1)
gt_op = tf.argmax(y_prim, 1)

p_mask = prim_mask

for j in file_indices:
    
    X_mini = None
    Y_mini = None

    lmd_tic = time.time()
    X_mini, Y_mini, mask_mini = load_mini_dataset(j) # todo Samarth add for mask - done
    print("at file index ", j)
    lmd_toc = time.time()
    total_time_taken_to_load_dataset += (lmd_toc-lmd_tic)
    
    rmb_tic = time.time()
    mini_batches = None
    mini_batches = create_random_mini_batches(X_mini, Y_mini, mask_mini, mini_batch_size = mini_batch_size) # todo Samarth add for mask
    rmb_toc = time.time()
    total_time_taken_to_create_mini_batches += (rmb_toc-rmb_tic)
    
    bat_ind = 0
    for mini_batch in mini_batches:
        #print("at minibatch ", bat_ind)
        bat_ind+=1
        batch_img = mini_batch[0] 
        batch_label = mini_batch[1]
        batch_mask = mini_batch[2]
        curr_batch_size = batch_img.shape[0]

        for i in range(curr_batch_size):
            if batch_mask[i] == 0.0:
                prim_nb_train += 1
            else:
                if batch_mask[i] == 1.0:
                    sec_nb_train += 1                           

        prim_nb_true_pred += sess.run(prim_true_pred, feed_dict={x: batch_img, y_: batch_label, mask: batch_mask,
                                                                  phase_train: False,
                                                                  keep_prob: 0.5})

        sec_nb_true_pred += sess.run(sec_true_pred,
                                        feed_dict={x: batch_img, y_: batch_label, mask: batch_mask,
                                                  phase_train: False,
                                                  keep_prob: 0.5})
        
        #op1, gt_op1, p_mask1, TP1, FP1, TN1, FN1 = sess.run([op, gt_op, p_mask, TP, FP, TN, FN], feed_dict={x: batch_img, y_: batch_label, mask: batch_mask,
        #                                          phase_train: False,
        #                                          keep_prob: 0.5})
        
        y_prim_op, y_sec_op = sess.run([y_prim_conv, y_sec_conv], feed_dict={x: batch_img, y_: batch_label, mask: batch_mask, phase_train: False, keep_prob: 0.5})

        pred = np.argmax(y_prim_op, axis=1)
        true = np.argmax(batch_label, axis=1)
        
        if(int(true) != int(pred) and int(batch_mask) == 0):
            #0-properly worn    1-improperly worn
            print(bat_ind, pred, true)
            denorm = (batch_img[0] * 255.0) + 128 
            rgb = cv2.cvtColor(np.uint8(denorm),cv2.COLOR_GRAY2RGB)
            resized_rgb = cv2.resize(rgb, (128,128)) 
            cv2.imwrite('./sample_images/check' +str(j) + "_" + str(bat_ind) + '.png',resized_rgb)


prim_train_accuracy = prim_nb_true_pred * 1.0 / prim_nb_train
sec_train_accuracy = sec_nb_true_pred * 1.0 / sec_nb_train

print('Primary task train accuracy: ' + str(prim_train_accuracy * 100))
print('Secondary task train accuracy: ' + str(sec_train_accuracy * 100))

Restoring existed model
INFO:tensorflow:Restoring parameters from /content/models/model.ckpt
OK
at file index  0
168 [1] [0]
206 [0] [1]
305 [0] [1]
382 [0] [1]
391 [0] [1]
409 [0] [1]
1012 [0] [1]
at file index  1
506 [0] [1]
747 [1] [0]
953 [0] [1]
1049 [1] [0]
1323 [0] [1]
1400 [0] [1]
at file index  2
87 [0] [1]
762 [0] [1]
935 [0] [1]
1103 [0] [1]
1311 [0] [1]
at file index  3
165 [0] [1]
408 [0] [1]
419 [0] [1]
464 [0] [1]
470 [0] [1]
599 [0] [1]
835 [1] [0]
836 [0] [1]
1083 [1] [0]
1198 [0] [1]
1306 [1] [0]
1362 [1] [0]
at file index  4
41 [0] [1]
150 [1] [0]
597 [0] [1]
638 [0] [1]
788 [0] [1]
1208 [0] [1]
1490 [0] [1]
at file index  5
545 [0] [1]
564 [0] [1]
688 [0] [1]
1137 [0] [1]
1230 [0] [1]
1308 [0] [1]
at file index  6
326 [0] [1]
555 [0] [1]
591 [0] [1]
618 [0] [1]
756 [0] [1]
1035 [0] [1]
1224 [0] [1]
1492 [0] [1]
at file index  7
616 [0] [1]
691 [0] [1]
880 [0] [1]
1118 [0] [1]
at file index  8
313 [0] [1]
700 [0] [1]
888 [0] [1]
1199 [0] [1]
1401 [0] [1]
at file inde

In [ ]:
op_2 = tf.argmax(y_sec_conv, 1)
gt_op_2 = tf.argmax(y_sec, 1)  

In [ ]:
#to delete
label_to_int = {"Mask":0,"Mask_Mouth_Chin":1,"Mask_Chin":2,"Mask_Nose_Mouth":3}
int_to_label = {0:"Mask",1:"Mask_Mouth_Chin",2:"Mask_Chin",3:"Mask_Nose_Mouth"}
def predict_single_image(file_index, img_index):
    filename_prefix = '/content/sample_data/test_part'#/content/drive/MyDrive/GT/BigDataProject/train_part0.npy
    filename = filename_prefix + str(file_index) + ".npy"
    X = np.load(filename, allow_pickle=True)
  
    mask_prim = 0.0
    mask_sec = 1.0

    sample = X[img_index]
        
    norm_x_prim = normalise(sample[0]) # todo check output once
    x_mini_data =[]
    y_mini_data=[]
    mask = []    
    mask_prim = 0.0
    mask_sec = 1.0
    label_int = sample[1]
    y_exp_prim = [0] * NUM_CLASSES
    if int_to_label[label_int]=="Mask":
        x_mini_data.append(norm_x_prim)
        y_exp_prim[label_int] = 1 # y_ = [1,0,0]
        y_mini_data.append(y_exp_prim)
        mask.append(mask_prim)
          
    else:
        x_mini_data.append(norm_x_prim)
        y_exp_prim[1] = 1 # y_ = [0,1,0]
        y_mini_data.append(y_exp_prim)
        mask.append(mask_prim)
    print(x_mini_data)
    x_data = np.array(x_mini_data)
    y_data = np.array(y_mini_data)
    mask_data = np.array(mask)
    print(x_data.type)
    print(y_data)
    print(mask_data)
    y_prim_op, y_sec_op = sess.run([y_prim_conv, y_sec_conv], feed_dict={x: x_data, y_: y_data, mask: mask_data, phase_train: False, keep_prob: 0.5})
    
    print(y_prim_op)
    print(y_sec_op)
    print(sample[0].shape)
    rgb = cv2.cvtColor(np.uint8(sample[0]),cv2.COLOR_GRAY2RGB)
    print(rgb.shape)
    resized_rgb = cv2.resize(rgb, (128,128)) #, interpolation = cv2.INTER_AREA
    #print(resized_grey.shape)
    cv2.imwrite('./check1.png',resized_rgb)
    
    #print(resized_rgb.shape)
    #cv2.imshow(resized_rgb)
    
    #pil_image=Image.fromarray(sample[0].squeeze(), 'L')
    #pil_image = pil_image.resize((256, 256), Image.ANTIALIAS)
    #pil_image.save('./chk1.png')
    #print(op_val_1, op_val_2)
    print(label_int)


In [ ]:
predict_single_image(0, 234)

In [ ]:
%pwd

'/content'